<a href="https://colab.research.google.com/github/MasahiroAraki/MLCourse/blob/master/04_statistical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. 識別 ー統計的手法ー

課題：スムージングを行わない方法でナイーブベイズ識別器の学習を行い、ゴルフデータ中の特定のデータに対する識別結果の確率を求め、その確率計算が正しいことを検算してください。

In [1]:
import numpy as np
import pandas as pd
import io
from sklearn import preprocessing
from sklearn.naive_bayes import CategoricalNB

ゴルフデータをpandasのデータフレーム形式で読み込みます。

In [2]:
weather_nominal = '''
outlook,temperature,humidity,windy,play
sunny,hot,high,FALSE,no
sunny,hot,high,TRUE,no
overcast,hot,high,FALSE,yes
rainy,mild,high,FALSE,yes
rainy,cool,normal,FALSE,yes
rainy,cool,normal,TRUE,no
overcast,cool,normal,TRUE,yes
sunny,mild,high,FALSE,no
sunny,cool,normal,FALSE,yes
rainy,mild,normal,FALSE,yes
sunny,mild,normal,TRUE,yes
overcast,mild,high,TRUE,yes
overcast,hot,normal,FALSE,yes
rainy,mild,high,TRUE,no
'''
df = pd.read_csv(io.StringIO(weather_nominal), sep=',')

scikit-learnのデータセット形式であるndarrayとして、パターン行列Xと正解yを取り出します。  

In [3]:
X = df.values[:,0:4]
y = df.values[:,-1]

特徴はOrdinalEncoderで数値ベクトルに変換します。
OrdinalEncoderはカテゴリ特徴を整数に置き換えます。

In [4]:
enc = preprocessing.OrdinalEncoder()
X_en = enc.fit_transform(X)

In [5]:
enc.categories_

[array(['overcast', 'rainy', 'sunny'], dtype=object),
 array(['cool', 'hot', 'mild'], dtype=object),
 array(['high', 'normal'], dtype=object),
 array([False, True], dtype=object)]

正解は、LabelEncoderでyesとnoをそれぞれ1と0に置き換えます。

In [6]:
le = preprocessing.LabelEncoder()
y_en = le.fit_transform(y)

In [7]:
le.classes_

array(['no', 'yes'], dtype=object)

ナイーブベイズ識別器の学習を行います。スムージングを行わないためには、引数alphaの値を0にします。

参考：
* CategoricalNB https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html

In [8]:
clf = CategoricalNB(alpha=0)
clf.fit(X_en, y_en)

CategoricalNB(alpha=0, class_prior=None, fit_prior=True)

学習した統計モデルの中身を見ます。各次元の情報が一つの行列で表されて、行方向がクラス、列方向が特徴の値です。データセット中の頻度が単純にカウントされているだけで、スムージングの情報は入っていません。

In [9]:
clf.category_count_

[array([[0., 2., 3.],
        [4., 3., 2.]]),
 array([[1., 2., 2.],
        [3., 2., 4.]]),
 array([[4., 1.],
        [3., 6.]]),
 array([[2., 3.],
        [6., 3.]])]

(sunny, hot, high, FALSE)の判定結果を出力します。

In [10]:
X_test = enc.transform([['sunny', 'hot', 'high', False]])
X_test 

array([[2., 1., 0., 0.]])

In [11]:
clf.predict_proba(X_test)

array([[0.79541735, 0.20458265]])

事前確率は p=9/14, n=5/14。各特徴値の確率は、たとえばoutlook=sunnyの場合、positive（行列の2行目) 9事例中sunnyが2事例（行列の3列目）なので、2/9となります。

In [12]:
p=(2/9)*(2/9)*(3/9)*(6/9)*(9/14)
n=(3/5)*(2/5)*(4/5)*(2/5)*(5/14)
p/(p+n)

0.204582651391162